In [1]:
import pandas as pd
import numpy as np
import requests
import seaborn as sns
import json
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns',400)
from sklearn.preprocessing import LabelEncoder

In [2]:
train1 = pd.read_csv('train.csv')
test = pd.read_csv('test3.csv')
columns_test = test.columns.tolist()
train = train1[columns_test]
#test.head()

In [3]:
cols_features = ['CO_PROVA_MT', 'TX_RESPOSTAS_MT']
cols_X = ['NU_INSCRICAO','CO_PROVA_MT', 'TX_RESPOSTAS_MT']
train = train[cols_X]
test = test[cols_X]
train['CO_PROVA_MT'] = train['CO_PROVA_MT'].replace({"50ce41dc05c4888b3cb778b060c044741b100565": 'X', "109b8441d32cb8eebe003cbbf21e90d1c9c4abfa": 'AMARELA',"a142bfd77e5c63a016a597142c900c63a516e76b":'AZUL','1e7f6441d4706e748f6b1fb2f0827d556e1e638d':'CINZA','749033e660788d57d2ab756fa867fb48b8b2258d':"ROSA", "6bb8bda6f43fc888e379a194c882838992c67ff9":"AZUL2","cb4e46c73dd1093e9a908091c849744d00ccaede":"CINZA2", "095a12f1de800fa4912ec9a7eae5a86fd9a79d4e":"AMARELA2",'e09570bab3ec777b3f24e0e99af8e92cb3b87346':"ROSA2" })
train = train.dropna()
test['CO_PROVA_MT'] = test['CO_PROVA_MT'].replace({"50ce41dc05c4888b3cb778b060c044741b100565": 'X', "109b8441d32cb8eebe003cbbf21e90d1c9c4abfa": 'AMARELA',"a142bfd77e5c63a016a597142c900c63a516e76b":'AZUL','1e7f6441d4706e748f6b1fb2f0827d556e1e638d':'CINZA','749033e660788d57d2ab756fa867fb48b8b2258d':"ROSA", "6bb8bda6f43fc888e379a194c882838992c67ff9":"AZUL2","cb4e46c73dd1093e9a908091c849744d00ccaede":"CINZA2", "095a12f1de800fa4912ec9a7eae5a86fd9a79d4e":"AMARELA2",'e09570bab3ec777b3f24e0e99af8e92cb3b87346':"ROSA2" })

# Test features

In [4]:
def limpeza_features_test(trainf1):

#função de limpeza
    def split_tx_features(trainf1):
        tx = trainf1['TX_RESPOSTAS_MT'].str.split(r"(.)", expand=True).iloc[:,:81]
        tx = tx.drop(np.arange(0,81,2).astype(int), axis=1)
        new = [136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175]
        tx.columns = new
        tx['NU_INSCRICAO'] = trainf1['NU_INSCRICAO']    
        return tx 
    train_feat = split_tx_features(trainf1)
    trainf1 = trainf1.drop(['TX_RESPOSTAS_MT'], axis=1)
    train_feat = train_feat.replace({".":"*"})

    return trainf1.merge(train_feat, on='NU_INSCRICAO')

In [5]:
test_features = limpeza_features_test(test)
#test_features.head()

In [6]:
enc = LabelEncoder()
for i in range(136,176):
    col_enc = i
    test_features[col_enc] = enc.fit_transform(test_features[i])

# Train Features

In [7]:
def limpeza_features_train(trainf1):

#função de limpeza
    def split_tx_features(trainf1):
        tx = trainf1['TX_RESPOSTAS_MT'].str.split(r"(.)", expand=True)
        tx = tx.drop(np.arange(0,91,2).astype(int), axis=1)
        new = [136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180]
        tx.columns = new
        tx['NU_INSCRICAO'] = trainf1['NU_INSCRICAO']
        return tx 
    train_feat = split_tx_features(trainf1)
    trainf1 = trainf1.drop(['TX_RESPOSTAS_MT'], axis=1)
    train_feat = train_feat.replace({".":"*"})

    return trainf1.merge(train_feat, on='NU_INSCRICAO')

In [8]:
train_features = limpeza_features_train(train)
#train_features.head()

In [9]:
enc = LabelEncoder()
for i in range(136,176):
    col_enc = i
    train_features[col_enc] = enc.fit_transform(train_features[i])

# Separando X e y de train_features e train_target

In [10]:
prova_am2 = train_features[train_features['CO_PROVA_MT'] == 'AMARELA2'].drop(['CO_PROVA_MT'], axis=1)
prova_az2 = train_features[train_features['CO_PROVA_MT'] == 'AZUL2'].drop(['CO_PROVA_MT'], axis=1)
prova_r2 = train_features[train_features['CO_PROVA_MT'] == 'ROSA2'].drop(['CO_PROVA_MT'], axis=1)
prova_c2= train_features[train_features['CO_PROVA_MT'] == 'CINZA2'].drop(['CO_PROVA_MT'], axis=1)
prova_az = train_features[train_features['CO_PROVA_MT'] == 'AZUL'].drop(['CO_PROVA_MT'], axis=1)
prova_am = train_features[train_features['CO_PROVA_MT'] == 'AMARELA'].drop(['CO_PROVA_MT'], axis=1)
prova_r = train_features[train_features['CO_PROVA_MT'] == 'ROSA'].drop(['CO_PROVA_MT'], axis=1)
prova_c = train_features[train_features['CO_PROVA_MT'] == 'CINZA'].drop(['CO_PROVA_MT'], axis=1)

In [11]:
cols_y = [176,177,178,179,180]
cols_x = ['NU_INSCRICAO', 176,177,178,179,180]

# Separando em 5 respostas

In [12]:
y_am = prova_am[cols_y].values.astype(str)
X_am = prova_am.drop(cols_x,axis=1).values
y_az = prova_az[cols_y].values.astype(str)
X_az = prova_az.drop(cols_x,axis=1).values
y_c = prova_c[cols_y].values.astype(str)
X_c = prova_c.drop(cols_x,axis=1).values
y_r = prova_r[cols_y].values.astype(str)
X_r = prova_r.drop(cols_x,axis=1).values
y_am2 = prova_am2[cols_y].values.astype(str)
X_am2 = prova_am2.drop(cols_x,axis=1).values
y_az2 = prova_az2[cols_y].values.astype(str)
X_az2 = prova_az2.drop(cols_x,axis=1).values
y_c2 = prova_c2[cols_y].values.astype(str)
X_c2 = prova_c2.drop(cols_x,axis=1).values
y_r2 = prova_r2[cols_y].values.astype(str)
X_r2 = prova_r2.drop(cols_x,axis=1).values

# Modelo

In [13]:
from seqlearn.hmm import MultinomialHMM
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

Using TensorFlow backend.


In [14]:
def hmm_train (X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=101)
    classifier = MultinomialHMM (decode='bestfirst', alpha=0.5)
    lens = [np.sum([len(X_train), len(y_train)]).astype(int),]
    classifier.fit(X_train, y_train, lengths=lens)
    y_pred = classifier.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print(score)

In [15]:
def rf_train (X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=101)
    rf = RandomForestClassifier(n_estimators = 450, criterion='entropy', random_state=101, max_features=None)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print(score)

In [16]:
def lg_train (X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=101)
    lg = LogisticRegressionCV(class_weight='balanced', multi_class='multinomial', solver='newton-cg')
    lg.fit(X_train, y_train)
    y_pred = lg.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print(score)

In [17]:
def bl_train (X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=101)
    bl = BernoulliNB(alpha=0.5)
    bl.fit(X_train, y_train)
    y_pred = bl.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print(score)

In [18]:
def ga_train (X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=101)
    ga = GaussianNB()
    ga.fit(X_train, y_train)
    y_pred = ga.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print(score)

In [19]:
def svc_train (X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=101)
    svc = SVC(C=1.0, tol=1e-10, cache_size=600, kernel='rbf')
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print(score)

In [135]:
hmm_train(X_am,y_am[:,0:1].ravel())
hmm_train(X_am,y_am[:,1:2].ravel())
svc_train(X_am,y_am[:,2:3].ravel())
svc_train(X_am,y_am[:,3:4].ravel())
hmm_train(X_am,y_am[:,4:5].ravel())

0.334106728538
0.337973704563
0.557617942769
0.262180974478
0.30626450116


In [153]:
rf_train(X_am,y_am[:,0:1].ravel())
rf_train(X_am,y_am[:,1:2].ravel())
rf_train(X_am,y_am[:,2:3].ravel())
rf_train(X_am,y_am[:,3:4].ravel())
rf_train(X_am,y_am[:,4:5].ravel())

0.311678267595
0.328692962104
0.555297757154
0.256767208043
0.297757153906


In [133]:
hmm_train(X_am,y_am[:,0:1].ravel())
hmm_train(X_am,y_am[:,1:2].ravel())
hmm_train(X_am,y_am[:,2:3].ravel())
hmm_train(X_am,y_am[:,3:4].ravel())
hmm_train(X_am,y_am[:,4:5].ravel())

0.334106728538
0.337973704563
0.556844547564
0.247486465584
0.30626450116


In [139]:
rf_train(X_az,y_az[:,0:1].ravel())
rf_train(X_az,y_az[:,1:2].ravel())
rf_train(X_az,y_az[:,2:3].ravel())
rf_train(X_az,y_az[:,3:4].ravel())
rf_train(X_az,y_az[:,4:5].ravel())

0.275098814229
0.286166007905
0.37233201581
0.302766798419
0.237154150198


In [22]:
rf_train(X_c,y_c[:,0:1].ravel())
rf_train(X_c,y_c[:,1:2].ravel())
rf_train(X_c,y_c[:,2:3].ravel())
rf_train(X_c,y_c[:,3:4].ravel())
rf_train(X_c,y_c[:,4:5].ravel())

0.274636510501
0.392568659128
0.273828756058
0.242326332795
0.264943457189


In [23]:
rf_train(X_r,y_r[:,0:1].ravel())
rf_train(X_r,y_r[:,1:2].ravel())
rf_train(X_r,y_r[:,2:3].ravel())
rf_train(X_r,y_r[:,3:4].ravel())
rf_train(X_r,y_r[:,4:5].ravel())

0.312753858652
0.260763606824
0.310316815597
0.313566206336
0.353371242892


In [24]:
rf_train(X_am2,y_am2[:,0:1].ravel())
rf_train(X_am2,y_am2[:,1:2].ravel())
rf_train(X_am2,y_am2[:,2:3].ravel())
rf_train(X_am2,y_am2[:,3:4].ravel())
rf_train(X_am2,y_am2[:,4:5].ravel())

0.371428571429
0.4
0.257142857143
0.142857142857
0.171428571429


In [25]:
rf_train(X_az2,y_az2[:,0:1].ravel())
rf_train(X_az2,y_az2[:,1:2].ravel())
rf_train(X_az2,y_az2[:,2:3].ravel())
rf_train(X_az2,y_az2[:,3:4].ravel())
rf_train(X_az2,y_az2[:,4:5].ravel())

0.3
0.55
0.175
0.525
0.225


In [26]:
rf_train(X_c2,y_c2[:,0:1].ravel())
rf_train(X_c2,y_c2[:,1:2].ravel())
rf_train(X_c2,y_c2[:,2:3].ravel())
rf_train(X_c2,y_c2[:,3:4].ravel())
rf_train(X_c2,y_c2[:,4:5].ravel())

0.297297297297
0.486486486486
0.378378378378
0.297297297297
0.324324324324


In [27]:
rf_train(X_r2,y_r2[:,0:1].ravel())
rf_train(X_r2,y_r2[:,1:2].ravel())
rf_train(X_r2,y_r2[:,2:3].ravel())
rf_train(X_r2,y_r2[:,3:4].ravel())
rf_train(X_r2,y_r2[:,4:5].ravel())

0.4375
0.4375
0.375
0.375
0.25


# Separando em cores e questões TEST

In [20]:
prova_test_am2 = test_features[test_features['CO_PROVA_MT'] == 'AMARELA2'].drop(['CO_PROVA_MT'], axis=1)
prova_test_az2 = test_features[test_features['CO_PROVA_MT'] == 'AZUL2'].drop(['CO_PROVA_MT'], axis=1)
prova_test_r2 = test_features[test_features['CO_PROVA_MT'] == 'ROSA2'].drop(['CO_PROVA_MT'], axis=1)
prova_test_c2 = test_features[test_features['CO_PROVA_MT'] == 'CINZA2'].drop(['CO_PROVA_MT'], axis=1)
prova_test_az = test_features[test_features['CO_PROVA_MT'] == 'AZUL'].drop(['CO_PROVA_MT'], axis=1)
prova_test_am = test_features[test_features['CO_PROVA_MT'] == 'AMARELA'].drop(['CO_PROVA_MT'], axis=1)
prova_test_r = test_features[test_features['CO_PROVA_MT'] == 'ROSA'].drop(['CO_PROVA_MT'], axis=1)
prova_test_c = test_features[test_features['CO_PROVA_MT'] == 'CINZA'].drop(['CO_PROVA_MT'], axis=1)

In [21]:
Xt_am = prova_test_am.drop(['NU_INSCRICAO'],axis=1).values
Xt_az = prova_test_az.drop(['NU_INSCRICAO'],axis=1).values
Xt_c = prova_test_c.drop(['NU_INSCRICAO'],axis=1).values
Xt_r = prova_test_r.drop(['NU_INSCRICAO'],axis=1).values
Xt_am2 = prova_test_am2.drop(['NU_INSCRICAO'],axis=1).values
Xt_az2 = prova_test_az2.drop(['NU_INSCRICAO'],axis=1).values
Xt_c2 = prova_test_c2.drop(['NU_INSCRICAO'],axis=1).values
Xt_r2 = prova_test_r2.drop(['NU_INSCRICAO'],axis=1).values

# Modelo Test

In [22]:
def hmm_test (X_train,y_train, X_test):
    classifier = MultinomialHMM (decode='bestfirst', alpha=0.5)
    lens = [np.sum([len(X_train), len(y_train)]).astype(int),]
    classifier.fit(X_train, y_train, lengths=lens)
    y_pred = classifier.predict(X_test)
    return y_pred

In [33]:
def rf_test (X_train,y_train, X_test):
    rf = RandomForestClassifier(n_estimators = 405, criterion='entropy', random_state=101)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    return y_pred

In [24]:
def svc_test (X_train,y_train, X_test):
    svc = SVC(C=1.0, tol=1e-10, cache_size=600, kernel='rbf')
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    return y_pred

In [25]:
cols = [1,2,3,4,5, 'NU_INSCRICAO']

In [79]:
prova_test_am[1] = hmm_test(X_am,y_am[:,0:1].ravel(), Xt_am)
prova_test_am[2] = hmm_test(X_am,y_am[:,1:2].ravel(), Xt_am)
prova_test_am[3] = svc_test(X_am,y_am[:,2:3].ravel(), Xt_am)
prova_test_am[4] = svc_test(X_am,y_am[:,3:4].ravel(), Xt_am)
prova_test_am[5] = hmm_test(X_am,y_am[:,4:5].ravel(), Xt_am)
prova_test_am = prova_test_am[cols]

prova_test_az[1] = rf_test(X_az,y_az[:,0:1].ravel(), Xt_az)
prova_test_az[2] = rf_test(X_az,y_az[:,1:2].ravel(), Xt_az)
prova_test_az[3] = rf_test(X_az,y_az[:,2:3].ravel(), Xt_az)
prova_test_az[4] = rf_test(X_az,y_az[:,3:4].ravel(), Xt_az)
prova_test_az[5] = rf_test(X_az,y_az[:,4:5].ravel(), Xt_az)
prova_test_az = prova_test_az[cols]

prova_test_c[1] = rf_test(X_c,y_c[:,0:1].ravel(), Xt_c)
prova_test_c[2] = rf_test(X_c,y_c[:,1:2].ravel(), Xt_c)
prova_test_c[3] = rf_test(X_c,y_c[:,2:3].ravel(), Xt_c)
prova_test_c[4] = hmm_test(X_c,y_c[:,3:4].ravel(), Xt_c)
prova_test_c[5] = hmm_test(X_c,y_c[:,4:5].ravel(), Xt_c)
prova_test_c = prova_test_c[cols]

prova_test_r[1] = rf_test(X_r,y_r[:,0:1].ravel(), Xt_r)
prova_test_r[2] = rf_test(X_r,y_r[:,1:2].ravel(), Xt_r)
prova_test_r[3] = rf_test(X_r,y_r[:,2:3].ravel(), Xt_r)
prova_test_r[4] = rf_test(X_r,y_r[:,3:4].ravel(), Xt_r)
prova_test_r[5] = rf_test(X_r,y_r[:,4:5].ravel(), Xt_r)
prova_test_r = prova_test_r[cols]

prova_test_am2[1] = rf_test(X_am2,y_am2[:,0:1].ravel(), Xt_am2)
prova_test_am2[2] = rf_test(X_am2,y_am2[:,1:2].ravel(), Xt_am2)
prova_test_am2[3] = rf_test(X_am2,y_am2[:,2:3].ravel(), Xt_am2)
prova_test_am2[4] = rf_test(X_am2,y_am2[:,3:4].ravel(), Xt_am2)
prova_test_am2[5] = rf_test(X_am2,y_am2[:,4:5].ravel(), Xt_am2)
prova_test_am2 = prova_test_am2[cols]

prova_test_az2[1] = rf_test(X_az2,y_az2[:,0:1].ravel(), Xt_az2)
prova_test_az2[2] = svc_test(X_az2,y_az2[:,1:2].ravel(), Xt_az2)
prova_test_az2[3] = rf_test(X_az2,y_az2[:,2:3].ravel(), Xt_az2)
prova_test_az2[4] = svc_test(X_az2,y_az2[:,3:4].ravel(), Xt_az2)
prova_test_az2[5] = rf_test(X_az2,y_az2[:,4:5].ravel(), Xt_az2)
prova_test_az2 = prova_test_az2[cols]

prova_test_c2[1] = rf_test(X_c2,y_c2[:,0:1].ravel(), Xt_c2)
prova_test_c2[2] = rf_test(X_c2,y_c2[:,1:2].ravel(), Xt_c2)
prova_test_c2[3] = rf_test(X_c2,y_c2[:,2:3].ravel(), Xt_c2)
prova_test_c2[4] = rf_test(X_c2,y_c2[:,3:4].ravel(), Xt_c2)
prova_test_c2[5] = rf_test(X_c2,y_c2[:,4:5].ravel(), Xt_c2)
prova_test_c2 = prova_test_c2[cols]

prova_test_r2[1] = rf_test(X_r2,y_r2[:,0:1].ravel(), Xt_r2)
prova_test_r2[2] = rf_test(X_r2,y_r2[:,1:2].ravel(), Xt_r2)
prova_test_r2[3] = rf_test(X_r2,y_r2[:,2:3].ravel(), Xt_r2)
prova_test_r2[4] = rf_test(X_r2,y_r2[:,3:4].ravel(), Xt_r2)
prova_test_r2[5] = rf_test(X_r2,y_r2[:,4:5].ravel(), Xt_r2)
prova_test_r2 = prova_test_r2[cols]

In [80]:
prova_test = pd.concat([prova_test_am, prova_test_az, prova_test_c, prova_test_r, prova_test_am2, prova_test_az2, prova_test_c2, prova_test_r2], join='outer')

In [81]:
prova_test['TX_RESPOSTAS_MT'] = prova_test[1] + prova_test[2] + prova_test[3] + prova_test[4] + prova_test[5] 
prova_test = prova_test.drop([1,2,3,4,5], axis=1)

# Juntando com prova_5

In [103]:
prova_test = prova_test.rename(index=str, columns={1: "176_", 2: "177_", 3:"178_", 4:"179_", 5:"180_"})
respostas_5 = pd.read_csv('respostas_5_total.csv')
respostas_5 = respostas_5.drop(['Unnamed: 0'],axis=1)

In [104]:
prova_5 = prova_test.merge(respostas_5, on=['NU_INSCRICAO'], how='left')
prova_5['176_'] = prova_5['176_'][prova_5['176'].isnull()] 
prova_5['177_'] = prova_5['177_'][prova_5['177'].isnull()]
prova_5['178_'] = prova_5['178_'][prova_5['178'].isnull()]
prova_5['179_'] = prova_5['179_'][prova_5['179'].isnull()]
prova_5['180_'] = prova_5['180_'][prova_5['180'].isnull()]
prova_5 = prova_5.fillna('')
prova_5['176'] = prova_5['176'] + prova_5['176_']
prova_5['177'] = prova_5['177'] + prova_5['177_']
prova_5['178'] = prova_5['178'] + prova_5['178_']
prova_5['179'] = prova_5['179'] + prova_5['179_']
prova_5['180'] = prova_5['180'] + prova_5['180_']

In [105]:
prova_5 = prova_5.drop(['176_','177_','178_','179_','180_'], axis=1)
prova_5['TX_RESPOSTAS_MT'] = prova_5['176'] + prova_5['177'] + prova_5['178'] + prova_5['179'] + prova_5['180']
prova_5 = prova_5.drop(['176','177','178','179','180'], axis=1)
prova_5.head()

,NU_INSCRICAO,TX_RESPOSTAS_MT
0,00aeb990ba424494c5d80e531a35a5ad95ca0974,ADCEE
1,cd48ebad3a851751af436c2ba6d35f1c8393a9bb,ADCAE
2,5bd4b9c26f80f06e494c338ebb3174a274f6a89c,ADCEE
3,5d13ccbc6542cf047eb88bf09debd265ee9f61a0,ADCAC
4,92ee9d462a6d69c8c5608a75a6dac95b2af01f3a,ABCAC


# Submissão

In [82]:
Final_summit = prova_test
dic = Final_summit.to_dict("records")
dicf = {
  "token": "e2579c92a6ab22c1f8b612128ac33198167127c1",
  "email": "tiago.shink@gmail.com",
  "answer": dic}

In [83]:
import requests
r = requests.get('https://api.codenation.com.br/v1/user/acceleration/data-science/challenge/enem-3/submit')
r = requests.post('https://api.codenation.com.br/v1/user/acceleration/data-science/challenge/enem-3/submit', json = dicf)

In [84]:
print(r.text)

{"score":32.10622710622711}



In [ ]:
{"score":32.11233211233211}
{"score":32.11843711843712}

In [ ]:
{"score":31.739926739926737} - reglin sem scale
{"score":31.12942612942613} - reglin com scale
{"score":31.86202686202686} - prova_5 - respostas certas separados por cor
{"score":31.336996336996336} - prova_5_cluster - respostas certas separadas por cluster
{"score":31.89255189255189} - respostas certas separadas por cor. Regressão feita com amostra total

In [ ]:
{"score":31.81318681318681} - 350
{"score":31.84371184371184} - 380
{"score":31.898656898656895} - 395
{"score":31.98412698412698} - 400
{"score":32.032967032967036} - 405
{"score":31.990231990231987} - 406
{"score":31.978021978021975} - 410
{"score":31.770451770451768} - 450
{"score":31.87423687423687} - 500



{"score":31.776556776556774} - mixed 405
